# Request Type Analysis

Look at the request type values from 311.  Questions to consider:

  - Counts
  - Spatial (NCs) distribution
  - Time to complete
  - Time to complete by service provider
  - Spatial (service region) distribution?
  - ...

Steps in this notebook:

1.  Setup
2.  Create geodataframe/dataframe from cleaned data and [census](https://data.lacity.org/Community-Economic-Development/Census-Data-by-Neighborhood-Council/nwj3-ufba)
3.  Examine the data
4.  Compute the measure
5.  Show measure as choropleth
6.  So what (next steps)

# 1 - Setup

In [ ]:
%run start.py
from utils import read_new311_shape, dt_to_object

# 2 - Get Data Files

Using two data files:

  - The Certified Neighborhoods cleaned up in NC-service-regions.ipynb
  - graffiti counts from 311


In [ ]:
%%time
graffiti_gdf = read_new311_shape('../data/311/graffiti.geojson.zip')

In [ ]:
graffiti_gdf.columns

In [ ]:
graffiti_gdf['owner'].value_counts()

In [ ]:
neighborhoods_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.shp')

neighborhoods_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

In [ ]:
neighborhoods_gdf.info()

# 3 - Quick Looksie

In [ ]:
graffiti_counts = graffiti_gdf['nc'].value_counts().to_frame().reset_index().rename(columns={'index': 'nc_id', 'nc': 'count'})

In [ ]:
graffiti_counts

In [ ]:
graffiti_merged = pd.merge(neighborhoods_gdf, graffiti_counts, how="left", on=["nc_id"])

In [ ]:
graffiti_merged

# 4 - Display the Choropleth

**Note** add legend for presentation charts.

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(34.05, -118.25), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

#map_display.add_control(LayersControl())
#map_display += nc_layer
map_display

refer to : https://www.youtube.com/watch?v=wjzAy_yLrdA

In [ ]:
from ipyleaflet import Choropleth, Map
from branca.colormap import linear
a_geojson = json.loads(graffiti_merged.to_json())

graffiti_density = dict(zip(graffiti_merged['name'].tolist(), graffiti_merged['count'].tolist()))
for i in a_geojson['features']:
    i['id'] = i['properties']['name']

layer = Choropleth(
                    geo_data=a_geojson,
                    choro_data=graffiti_density,
                    colormap=linear.YlOrRd_09, #linear.Blues_05,
                    style={'fillOpacity': 1.0, "color":"black"},)
                    #key_on="name")

map_display.add_layer(layer)

I need to revisit a tooltip type popup.  For now this will work.

In [ ]:
geo_json = GeoJSON(
    data=a_geojson,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    name='NCs'
)

html = HTML('''Hover over a district''')
html.layout.margin = '0px 20px 20px 20 px'
control = WidgetControl(widget=html, position='bottomright')

def update_html(feature, **kwargs):
    html.value = '''<h3><b>NC: {}</b></h3>
                    <h4>Count: {}'''.format(feature['properties']['name'],
                                                           feature['properties']['count'])
    
map_display.add_control(control)  # does += work for this?

layer.on_hover(update_html)

# 6 - So What?

I say this tounge in cheeck.  Things to think about:

  1. Should we examine measures besides total population?
  2. Does it make sense to extend the 311 data as we did with the service regions?
  3. Do we just use this to select an NC then query 311 (or ...)?
  
